In [1]:
%cd ..

/home/zarizky/projects/neural-autoregressive-object-co-occurrence


In [2]:
import json
from functools import partial

import networkx as nx
import numpy as np
import pandas as pd
from IPython.display import Image
from pgmpy.estimators import BayesianEstimator as BE
from pgmpy.estimators import TreeSearch
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
from pgmpy.metrics import BayesianModelProbability
from pgmpy.models import BayesianNetwork
from tqdm.contrib.concurrent import process_map

import utils

In [3]:
df_train = pd.read_csv("dataset/coco2017-cooccurences-train.csv").iloc[:, 8:]
df_valid = pd.read_csv("dataset/coco2017-cooccurences-valid.csv").iloc[:, 8:]

df_train.columns = [
    col.split("-")[0].replace("[", "").replace("]", "").upper()
    for col in df_train.columns
]

df_valid.columns = df_train.columns

In [4]:
search = TreeSearch(df_train)
dag = search.estimate(estimator_type="chow-liu")

Building tree:   0%|          | 0/3160.0 [00:00<?, ?it/s]

In [5]:
model = BayesianNetwork(dag)
model.fit(df_train, estimator=BE)

In [6]:
engine = BayesianModelProbability(model)
engine.log_probability(df_train).mean()

-13.452676145989034

In [7]:
model_new = BayesianNetwork(dag)
D = df_train.values.max() + 1

eps = 1e-1
for cpd in model.get_cpds():
    evi = cpd.get_evidence()
    var = cpd.variable
    if len(evi) <= 0:
        src = list(cpd.no_to_name.values())[0]
        table_old = cpd.get_values()
        table_new = np.zeros((D, 1))
        for i in src:
            table_new[src[i], :] = table_old[i]
    else:
        src, tgt = cpd.no_to_name.values()
        table_old = cpd.get_values()
        table_new = np.zeros([D] * 2)
        for i in src:
            for j in tgt:
                table_new[src[i], tgt[j]] = table_old[i, j]

    table_new *= len(df_train)
    table_new += eps
    table_new /= table_new.sum(0)

    if len(evi) <= 0:
        model_new.add_cpds(TabularCPD(var, table_new.shape[0], table_new))
    else:
        model_new.add_cpds(
            TabularCPD(var, table_new.shape[0], table_new, evi, [table_new.shape[-1]])
        )

/tmp/ipykernel_2361169/2703822786.py:30: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  TabularCPD(var, table_new.shape[0], table_new, evi, [table_new.shape[-1]])


In [100]:
dag_cpy = nx.DiGraph()
dag_cpy.add_nodes_from(dag.nodes)
dag_cpy.add_edges_from(dag.edges)
dag_cpy = nx.relabel_nodes(dag_cpy, {k: k.replace(" ", "\n") for k in dag.nodes})

pgv = nx.nx_agraph.to_agraph(dag_cpy)

independent = [
    node
    for node in dag_cpy.successors("PERSON")
    if len(list(dag_cpy.successors(node))) <= 0
]

idp_ = [[] for _ in range(len(independent) // 12)]

for i in range(len(independent)):
    idp_[i % 2].append(independent[i])

pgv_idp = pgv.add_subgraph(independent, name="idp", rankdir="LR")

for sub in idp_:
    for i in range(1, len(sub)):
        pgv_idp.add_edge(sub[i - 1], sub[i], style="invis")

pgv.graph_attr["splines"] = "polyline"
pgv.graph_attr["overlap"] = "false"
pgv.graph_attr["dpi"] = 512
pgv.graph_attr["margin"] = "0"
pgv.node_attr["shape"] = "box"
pgv.draw("outputs/chow_liu_tree/graph.png", prog="dot")
pgv.draw("outputs/chow_liu_tree/graph.pdf", prog="dot")

display(Image("outputs/chow_liu_tree/graph.png"))

In [9]:
model_new.check_model()

total_params = 0
for cpd in model_new.get_cpds():
    total_params += cpd.values.size
total_params

66468

In [10]:
batch_size = 128
batch_train = [
    df_train.iloc[i : i + batch_size] for i in range(0, len(df_train), batch_size)
]
batch_valid = [
    df_valid.iloc[i : i + batch_size] for i in range(0, len(df_valid), batch_size)
]

log_posterior = partial(utils.pgmpy.log_posterior, model=model_new)

engine = BayesianModelProbability(model_new)

total_train_ll = engine.log_probability(df_train).sum()
avg_train_ll = total_train_ll / len(df_train)
train_bic = total_train_ll - total_params / 2 * np.log(len(df_train))

total_valid_ll = engine.log_probability(df_valid).sum()
avg_valid_ll = total_valid_ll / len(df_valid)

total_train_lp = sum(process_map(log_posterior, batch_train))
avg_train_lp = total_train_lp / len(df_train)

total_valid_lp = sum(process_map(log_posterior, batch_valid))
avg_valid_lp = total_valid_lp / len(df_valid)

print(f"Avg. Train NLL: {-avg_train_ll:.4f}")
print(f"Avg. Valid NLL: {-avg_valid_ll:.4f}")
print(f"Avg. Train NLP: {-avg_train_lp:.4f}")
print(f"Avg. Valid NLP: {-avg_valid_lp:.4f}")
print(f"BIC: {train_bic:,.4f}")

  0%|          | 0/925 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

Avg. Train NLL: 13.4535
Avg. Valid NLL: 13.7067
Avg. Train NLP: 12.5542
Avg. Valid NLP: 12.9422
BIC: -1,979,573.5735


In [11]:
with open("outputs/overall/chow_liu_tree.json", "w", encoding="utf-8") as file:
    json.dump(
        dict(
            model="Chow-Liu Tree",
            kind="Structured Model",
            total_parameters=total_params,
            avg_train_nll=-avg_train_ll,
            avg_test_nll=-avg_valid_ll,
            avg_train_nlp=-avg_train_lp,
            avg_test_nlp=-avg_valid_lp,
            neg_bic=-train_bic,
        ),
        file,
        ensure_ascii=False,
        indent=4,
    )

In [13]:
sum(len(df) for df in batch_train) == len(df_train)

True